In [1]:
from pdf2image import convert_from_path
from dotenv import load_dotenv
load_dotenv()
import os
doc_path = os.getenv('pdf_coop')
pages = convert_from_path(doc_path)

In [2]:
import cv2
import numpy as np

def deskew(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.bitwise_not(gray)
    coords = np.column_stack(np.where(gray > 0))
    angle = cv2.minAreaRect(coords)[-1]
    
    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle

    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)

    return rotated

In [3]:
import pytesseract

def extract_text_from_image(image):
    text = pytesseract.image_to_string(image)
    return text

In [4]:
extracted_text = []

for page in pages:
    # Step 2: Preprocess the image (deskew)
    preprocessed_image = deskew(np.array(page))

    # Step 3: Extract text using OCR
    text = extract_text_from_image(preprocessed_image)
    extracted_text.append(text)

In [11]:
from pprint import pprint
pprint(extracted_text[4])

('DEPOSIT\n'
 '\n'
 'PRODUCTS\n'
 '\n'
 'LOCAL CURRENCY DEPOSIT PRODUCTS\n'
 '\n'
 'ORDINARY\n'
 '\n'
 'SAVING ACCOUNT\n'
 'PRODUCT OVERVIEW:\n'
 '\n'
 'Is an interest-bearing option designed to cater\n'
 'to a diverse range of customers with a modest\n'
 'initial deposit of just Birr SO.\n'
 '\n'
 'Key Features\n'
 '\n'
 '» It earns a competitive 7% interest\n'
 'rate, compounded.\n'
 '\n'
 'e It can be opened without initial\n'
 'balance, allowing customer to\n'
 'establish a banking relationship\n'
 'effortlessly.\n'
 '\n'
 '» Flexible account ownership\n'
 'options: individual, joint, trust,\n'
 'legalized cooperatives, and organi-\n'
 'zations.\n'
 '\n'
 'BENEFITS\n'
 '\n'
 '» Unrestricted deposits and with-\n'
 'drawals, subject only to regulations\n'
 'imposed by relevant authorities.\n'
 '\n'
 '* Convenience of seamless transac-\n'
 'tions through various channels,\n'
 'including ATM/POS cards, internet\n'
 'banking, and mobile services.\n'
 '\n'
 '» Zero transaction fees, prov

In [6]:

from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_text(extracted_text)
splits[0]

TypeError: expected string or bytes-like object, got 'list'